In [75]:
import json
from glob import glob
from tqdm import tqdm

In [89]:
http_errors = [
        "400 Bad Request", "401 Unauthorized", "402 Payment Required", "403 Forbidden", "404 Not Found",
        "405 Method Not Allowed", "406 Not Acceptable", "407 Proxy Authentication Required", "408 Request Timeout",
        "409 Conflict", "410 Gone", "411 Length Required", "412 Precondition Failed", "413 Payload Too Large",
        "414 URI Too Long", "415 Unsupported Media Type", "416 Range Not Satisfiable", "417 Expectation Failed",
        "418 I'm a teapot", "421 Misdirected Request", "422 Unprocessable Entity", "423 Locked", "424 Failed Dependency",
        "425 Too Early", "426 Upgrade Required", "428 Precondition Required", "429 Too Many Requests",
        "431 Request Header Fields Too Large", "451 Unavailable For Legal Reasons", "500 Internal Server Error",
        "501 Not Implemented", "502 Bad Gateway", "503 Service Unavailable", "504 Gateway Timeout",
        "505 HTTP Version Not Supported", "506 Variant Also Negotiates", "507 Insufficient Storage",
        "508 Loop Detected", "510 Not Extended", "511 Network Authentication Required"
    ]

rejected = [
    'Internal Server Error',
    '404',
    '__NOEDITSECTION__',
    'enter your username and password',
    'Login',
    'forgotten your password',
    'cookies enabled',
    'sign in',
    'tentang kami',
    'skip to content',
    'hubungi kami',
    'laman utama',
    'enable JavaScript in your browser.',
    'The page cannot be displayed',
    'site or edit the error_page',
    'Hakcipta terpelihara',
    'Copyright ©'
]

rejected.extend(http_errors)

In [80]:
files = sorted(glob('crawl-my-website/snapshot/*.json'))

In [114]:
!rm hf-datasets/dedupe-datasets/snapshot.jsonl

In [115]:
processed = set()
with open('hf-datasets/raw-datasets/snapshot.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                if l['url'] in processed:
                    continue

                splitted = l['data'].split('\n')
                splitted = [s for s in splitted if len(s) > 50]
                splitted = [s.strip() for s in splitted if all([r not in s for r in rejected])]
                if len(splitted):
                    data = {
                        'url': l['url'],
                        'text': splitted
                    }
                    fopen_l.write(f'{json.dumps(data)}\n')
                    fopen_l.flush()
                    
                processed.add(l['url'])

100%|██████████| 348/348 [03:33<00:00,  1.63it/s]


In [116]:
len(processed)

428982